# Gesture Recognition
In this project, we are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. 

In [14]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [15]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, we read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. We can keep increasing the batch size until the machine throws an error.

In [16]:
# server execution
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 28 #experiment with the batch size

## Generator
This is one of the most important part of the code. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. We have to experiment with `img_idx`, `y`,`z` and normalization to get high accuracy.

In [17]:
def generator_84x84(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [10,12,14,16,17,18,19,20,21,22,23,24,25,26,27,28] # List of image numbers we want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,16,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,16,84,84,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
  
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [ ]:
def generator_120x120(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [4,6,8,10,12,14,16,18,19,20,21,22,23,25,27,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,16,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(120,120)).astype(np.float32)
                    else:
                        image = imresize(image,(120,120)).astype(np.float32)
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],5)) / (np.percentile(image[:,:,0],95) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],5)) / (np.percentile(image[:,:,1],95) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],5)) / (np.percentile(image[:,:,2],95) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,16,84,84,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(120,120)).astype(np.float32)
                    else:
                        image = imresize(image,(120,120)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],5)) / (np.percentile(image[:,:,0],95) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],5)) / (np.percentile(image[:,:,1],95) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],5)) / (np.percentile(image[:,:,2],95) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
  
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

Generator for LSTM kept separately as the input for LSTM would be 3-D

In [18]:
def generator_lstm_120x120(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,2,3,5,7,9,11,13,15,17,19,21,23,25,27,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,16,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(120,120)).astype(np.float32)
                    else:
                        image = imresize(image,(120,120)).astype(np.float32)
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            #reshaping batch_data as 3-dimensional for feeding into LSTM
            yield batch_data.reshape(batch_size,16,120*120*3), batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,16,120,120,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(120,120)).astype(np.float32)
                    else:
                        image = imresize(image,(120,120)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
  
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            #reshaping batch_data as 3-dimensional for feeding into LSTM
            yield batch_data.reshape(len(t)%batch_size,16,120*120*3), batch_labels

In [ ]:
def generator_cropped(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,5,7,9,12,14,16,18,20,22,24,26,28] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,16,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    
                    if image.shape[1] == 160:
                        image = imresize(image[:,10:120,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,16,84,84,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,10:120,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
  
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [ ]:
def generator_cropped1(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,5,7,9,12,14,16,18,20,22,24,26,28] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,16,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,16,84,84,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
  
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [ ]:
def generator_final(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,5,7,9,12,14,16,18,20,22,24,26,28] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,16,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,16,84,84,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
  
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [19]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model 1 - variation 1
Using `Conv3D` and `MaxPooling3D`

In [20]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [21]:
model1 = Sequential()
model1.add(Conv3D(28, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model1.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model1.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model1.add(Flatten())
model1.add(Dropout(0.5))
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(5, activation='softmax'))

Now that we have written the model, the next step is to `compile` the model with `optimiser`

In [22]:
optimiser = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.5, nesterov=True) #write your optimizer
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 16, 84, 84, 28)    2296      
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 84, 84, 28)    112       
_________________________________________________________________
activation_4 (Activation)    (None, 16, 84, 84, 28)    0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 8, 42, 42, 28)     0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 8, 42, 42, 64)     48448     
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 42, 42, 64)     256       
_________________________________________________________________
activation_5 (Activation)    (None, 8, 42, 42, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [23]:
train_generator = generator_84x84(train_path, train_doc, batch_size)
val_generator = generator_84x84(val_path, val_doc, batch_size)

In [24]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\keras\callbacks.py:928: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` insted.
  warnings.warn('`epsilon` argument is deprecated and '


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Epoch 1/30 ./Project_data/train ; batch size = 28



C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


## Model 2 Variation 1
Increased the No. of Conv3D layers with lower learning rate

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


model = Sequential()
model.add(Conv3D(28, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = optimizers.SGD(lr=0.003, decay=1e-6, momentum=0.5, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator_84x84(train_path, train_doc, batch_size)
val_generator = generator_84x84(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 2 Variation 2
Changed Cropping ratio of the Image and trained with higher batch size and lower learning rate

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


model = Sequential()
model.add(Conv3D(42, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(512, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator_cropped(train_path, train_doc, batch_size)
val_generator = generator_cropped(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 2 Variation 3
Changed Cropping ratio further for the Image keeping the rest same

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


model = Sequential()
model.add(Conv3D(42, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(512, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator_cropped1(train_path, train_doc, batch_size)
val_generator = generator_cropped1(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 2 Variation 4
Changed Cropping ratio to 20:140 of the Image keeping the rest same

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


model = Sequential()
model.add(Conv3D(42, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,2)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(512, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator_final(train_path, train_doc, batch_size)
val_generator = generator_final(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 2
Using `LSTM` and `TimeDistributed` `Dense`

In [ ]:
model2 = Sequential()
model2.add(LSTM(256,dropout=0.2,input_shape=(16, 120*120*3)))
model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(0.5))
model2.add(TimeDistributed(Dense(5, activation='softmax')))

In [ ]:
sgd = SGD(lr=0.005, decay = 1e-6, momentum=0.9, nesterov=True)
model2.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print (model2.summary())

Using specific `generator` code for `LSTM`

In [ ]:
train_generator = generator_lstm_120x120(train_path, train_doc, batch_size)
val_generator = generator_lstm_120x120(val_path, val_doc, batch_size)

In [ ]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 3
Using `LSTM` and `Dense`

In [ ]:
model3 = Sequential()
model3.add(LSTM(256,dropout=0.2,recurrent_dropout=0.2,input_shape=(16, 120*120*3)))
model3.add(Dense(1024, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(5, activation='softmax'))

In [ ]:
sgd = SGD(lr=0.005, decay = 1e-6, momentum=0.9, nesterov=True)
model3.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print (model3.summary())

Using specific `generator` code for `LSTM`

In [ ]:
train_generator = generator_lstm_120x120(train_path, train_doc, batch_size)
val_generator = generator_lstm_120x120(val_path, val_doc, batch_size)

In [ ]:
model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 4 - variation 1
Using `TimeDistributed` `Conv2D` and `MaxPooling2D` followed by `LSTM` and `Dense`

In [ ]:
input_shape = (16,120,120, 3)
hg_classes = 5

model4 = Sequential()

model4.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),
    activation='relu', padding='same'), input_shape=input_shape))
model4.add(TimeDistributed(Conv2D(32, (3,3),
    kernel_initializer="he_normal", activation='relu')))
model4.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model4.add(TimeDistributed(Conv2D(64, (3,3),
    padding='same', activation='relu')))
model4.add(TimeDistributed(Conv2D(64, (3,3),
    padding='same', activation='relu')))
model4.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model4.add(TimeDistributed(Conv2D(128, (3,3),
    padding='same', activation='relu')))
model4.add(TimeDistributed(Conv2D(128, (3,3),
    padding='same', activation='relu')))
model4.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model4.add(TimeDistributed(Conv2D(256, (3,3),
    padding='same', activation='relu')))
model4.add(TimeDistributed(Conv2D(256, (3,3),
    padding='same', activation='relu')))
model4.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model4.add(TimeDistributed(Conv2D(512, (3,3),
    padding='same', activation='relu')))
model4.add(TimeDistributed(Conv2D(512, (3,3),
    padding='same', activation='relu')))
model4.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model4.add(TimeDistributed(Flatten()))

model4.add(Dropout(0.5))
model4.add(LSTM(256, return_sequences=False, dropout=0.5))
model4.add(Dense(hg_classes, activation='softmax'))


In [ ]:
sgd = SGD(lr=0.005, decay = 1e-6, momentum=0.9, nesterov=True)
model4.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print(model4.summary())

In [ ]:
train_generator = generator_120x120(train_path, train_doc, batch_size)
val_generator = generator_120x120(val_path, val_doc, batch_size)

In [ ]:
model4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 4 - variation 2
## Using image transformation in `Generator`
Creating methods for image processing - `affine` and `edge` transformation

In [ ]:
def getRandomAffine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    A = np.float32([[1, 0, dx], [0, 1, dy]])
    return A
def affine_change(image, A):
    return cv2.warpAffine(image, A, (image.shape[0], image.shape[1]))
def edge_detect(image):
    edge = np.zeros((image.shape[0], image.shape[1], image.shape[2]))
    edge[:,:,0] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,0],(3,3),0),cv2.CV_64F)
    edge[:,:,1] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,1],(3,3),0),cv2.CV_64F)
    edge[:,:,2] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,2],(3,3),0),cv2.CV_64F)
    return edge

In [ ]:
def generator_imgtrans_120x120(source_path, folder_list, batch_size, affine=False, edge=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [4,6,8,10,12,14,16,18,19,20,21,22,23,25,27,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,16,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(120,120))
                    else:
                        image = imresize(image,(120,120))
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                        
                    if edge:
                        image = edge_detect(image)

                    if affine:
                        image = affine_change(image, getRandomAffine())

                    image = image.astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,16,120,120,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(120,120))
                    else:
                        image = imresize(image,(120,120))

                    if edge:
                        image = edge_detect(image)
                        
                    if affine:
                        image = affine_change(image, getRandomAffine())
                    
                    image = image.astype(np.float32)

                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
  
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

Doing both `affine` and `edge` transformation

In [ ]:
train_generator = generator_imgtrans_120x120(train_path, train_doc, batch_size, affine=True, edge=True)
val_generator = generator_imgtrans_120x120(val_path, val_doc, batch_size, affine=True, edge=True)

In [ ]:
model4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 4 - variation 3
Doing only `edge` transformation

In [ ]:
train_generator = generator_imgtrans_120x120(train_path, train_doc, batch_size, affine=False, edge=True)
val_generator = generator_imgtrans_120x120(val_path, val_doc, batch_size, affine=False, edge=True)

In [ ]:
model4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 1 - variation 2
Using image transformation with `Conv3D` and `MaxPooling3D`
Using 120x120 dimension causes OOM error. So, keeping 84x84 resolution

In [ ]:
def generator_imgtrans_84x84(source_path, folder_list, batch_size, affine=False, edge=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [4,6,8,10,12,14,16,18,19,20,21,22,23,25,27,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,16,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(84,84))
                    else:
                        image = imresize(image,(84,84))
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                        
                    if edge:
                        image = edge_detect(image)

                    if affine:
                        image = affine_change(image, getRandomAffine())

                    image = image.astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,16,84,84,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:120,:],(84,84))
                    else:
                        image = imresize(image,(84,84))

                    if edge:
                        image = edge_detect(image)
                        
                    if affine:
                        image = affine_change(image, getRandomAffine())
                    
                    image = image.astype(np.float32)

                    batch_data[folder,idx,:,:,0] =  (image[:,:,0] - np.percentile(image[:,:,0],25)) / (np.percentile(image[:,:,0],75) - np.percentile(image[:,:,0],5))    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],25)) / (np.percentile(image[:,:,1],75) - np.percentile(image[:,:,1],5))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],25)) / (np.percentile(image[:,:,2],75) - np.percentile(image[:,:,2],5))   #normalise and feed in the image
  
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

Using only `edge` transformation

In [ ]:
train_generator = generator_imgtrans_84x84(train_path, train_doc, batch_size,affine=False,edge=True)
val_generator = generator_imgtrans_84x84(val_path, val_doc, batch_size,affine=False,edge=True)

In [ ]:
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model 1 - variation 3
Using both `affine` and `edge` transformation

In [ ]:
train_generator = generator_imgtrans_84x84(train_path, train_doc, batch_size,affine=True,edge=True)
val_generator = generator_imgtrans_84x84(val_path, val_doc, batch_size,affine=True,edge=True)

In [ ]:
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)